In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\jlsin\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


## Visit the NASA mars news site

In [3]:
# Visit the Mars news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser html to a soup object
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [5]:
# display the current title content
slide_elem(class_='content_title')

[<div class="content_title">NASA's Push to Save the Mars InSight Lander's Heat Probe</div>]

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find(class_='content_title').text
news_title

"NASA's Push to Save the Mars InSight Lander's Heat Probe"

In [7]:
news_p = slide_elem.find(class_='article_teaser_body').text
news_p

"The scoop on the end of the spacecraft's robotic arm will be used to 'pin' the mole against the wall of its hole."

## JPL Space Images Featured Image

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_link = browser.find_by_tag('button')[1]
full_image_link.click()

In [10]:
# Parse the resulting html with soup
# Convert the browser html to a soup object
html = browser.html
space_image = soup(html, 'html.parser')

In [11]:
# find the relative image url
img_url_rel = space_image.find_all('img')[1]['src']
img_url_rel

'image/featured/mars1.jpg'

In [12]:
# use the base url to create an absolute url
featured_img_url = url+img_url_rel
featured_img_url

'https://spaceimages-mars.comimage/featured/mars1.jpg'

## Mars Facts

In [13]:
# visit url
url = 'https://galaxyfacts-mars.com/'
browser.visit(url)

In [14]:
# find table information using 'pd.read_html'
match = "Mars - Earth Comparison"
mars_list = pd.read_html(url, match=match)
df = mars_list[0]
df

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# rename columns
df = df.set_axis(['Description', 'Mars', 'Earth'], axis=1, inplace=False)
df

,Description,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
# set Description column as index
df = df.set_index('Description')
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [17]:
# convert df to a HTML table string
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Hemispheres

In [18]:
url = 'https://marshemispheres.com/'

browser.visit(url)

In [19]:
# Create a list to hold the images and titles.
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css('a.product-item img')

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    
    # hemisphere info dictionary
    hemisphere_info = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample = browser.links.find_by_text('Sample').first
    hemisphere_info['img_url'] = sample['href']
    
    # Get Hemisphere title
    hemisphere_info['title'] = browser.find_by_css('h2.title').text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere_info)
    
    # Finally, we navigate backwards
    browser.back()

In [20]:
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [21]:
browser.quit()